In [ ]:
%reset -f

# Voxel painter in pythreejs
Based on: [voxel painter](http://threejs.org/examples/webgl_interactive_voxelpainter.html) ([source](https://github.com/mrdoob/three.js/blob/master/examples/webgl_interactive_voxelpainter.html))

TODO: 
    - Have rollOver helper snap to other voxel face
    - Delete voxel when shift-clicked

In [ ]:
from __future__ import division, print_function

In [ ]:
from pythreejs import *
from IPython.display import display
from ipywidgets import HTML
from traitlets import link

In [ ]:
import random
import math
import numpy as np

### Geometry definitions

In [ ]:
csize = 50 # size of voxel
nx = ny = 10
stepx, stepy = csize, csize
sizex, sizey = nx * csize, ny * csize

### Helper functions

In [ ]:
def normalize(list):
    """ 
    Normalize vector list 
    """
    return [x/sum(list) for x in list]

def rotation_matrix(angle, axis='x'):
    """ 
    Return rotation matrix as list of rows used in the
    Object3d.quaternion_from_rotation() class method
    """
    if axis in ['x','y','z']:
        sin = math.sin(angle)
        cos = math.cos(angle)
        # counter-clockwise rotation in yz-plane
        if axis is 'x':
            return [1, 0, 0, 0, cos, -sin, 0, sin, cos]
        # counter-clockwise rotation in xz-plane
        elif axis is 'y':
            return [cos, 0, sin, 0, 1, 0, -sin, 0, cos]
        # counter-clockwise rotation in xy-plane
        elif axis is 'z':
            return [cos, -sin, 0, sin, cos, 0, 0, 0, 1]
    else:
        raise ValueError('Cannot rotate about %s axis' % axis)

### Scene objects:

#### Voxels

In [ ]:
cube_tex = ImageTexture(
    imageUri = 'textures/square-outline-textured.png'
)

cube_geo = BoxGeometry(
    width = csize,
    height = csize,
    depth = csize
)

cube_mat = MeshLambertMaterial(
    color = '#feb74c',
    shading = 'FlatShading',
    map = cube_tex
)

#### Roll-over helper

In [ ]:
rollOver_geo = cube_geo

rollOver_mat = MeshBasicMaterial(
    color = '#ff0000',
    opacity = 0.5,
    transparent = True
)

rollOver_point = Mesh(
    geometry = rollOver_geo,
    material = rollOver_mat,
    visible = False,
)

#### Surface

In [ ]:
surf_geo = SurfaceGeometry(
    z = [0] * (nx + 1) * (ny + 1),
    width = sizex,
    height = sizey,
    width_segments = nx,
    height_segments = ny,
)

In [ ]:
surf_grid = SurfaceGrid(
    geometry = surf_geo,
    material = LineBasicMaterial(
        color = '#000000',
        opacity = 0.2,
        transparent = True,
    ),
)

surface = Mesh(
    geometry = surf_geo,
    material = MeshBasicMaterial(
        color = '#00ff00',
        opacity = 0.3,
        transparent = True,
        side = 'DoubleSide',
        
    ),
    visible = True
)

m = rotation_matrix(-math.pi/2)
surface.setRotationFromMatrix(m)
surf_grid.setRotationFromMatrix(m)

### Pickers (raycasting)

In [ ]:
pickable_objects = Group()
pickable_objects.add(surface)

In [ ]:
click_picker = Picker(
    controlling = pickable_objects,
    event = 'dblclick'
)
mousemove_picker = Picker(
    controlling = pickable_objects,
    event = 'mousemove'
)

In [ ]:
def map_to_grid(value):
    """
    Convert continous to discrete coordinates
    """
    # limit position to positive y-axis
    if value[1] < 0: 
        value[1] = float(0)
        
    # limit to discrete steps based on cube size
    pos = [int(x//csize*csize+csize/2) for x in value]

    # if block already exist at this position, shift up
    while tuple(pos) in objects.keys():
        pos[1] += csize
    return pos

def map_to_voxel_side(voxel, face):
    """
    Convert continous to discrete coordinates
    """
    # limit to discrete steps based on cube size
    pos = list(voxel.position)
    print(face)
    if face in (0, 1):
        pos[0] += csize
    elif face in (2, 3):
        pos[0] -= csize
    elif face in (4, 5):
        pos[1] += csize
    elif face in (6, 7):
        pos[1] -= csize
    elif face in (8, 9):
        pos[2] += csize
    elif face in (10, 11):
        pos[2] -= csize

    # if block already exist at this position, mark as invalid
    while tuple(pos) in objects.keys():
        return None
    return pos

In [ ]:
objects = {} # contains all voxels added to the scene

def on_surf_click(change):
    """
    Create new object when mouse is clicked on surface
    """
    # convert position to discrete coordinates
    pos = map_to_grid(change.owner.point)
        
    # create new object
    voxel = Mesh(
        geometry = cube_geo,
        material = cube_mat,
        position = pos
    )
    
    # add new object to scene and object list
    pickable_objects.add(voxel)
    objects[tuple(map(int, voxel.position))] = voxel
    
def on_voxel_click(change):
    """
    Create new object when mouse is clicked on voxel
    """
    old_voxel = change.owner.object
    if change.owner.modifiers[0] is True:
        pickable_objects.remove(old_voxel)
        del objects[tuple(map(int, old_voxel.position))]
        return
    # convert position to discrete coordinates
    pos = map_to_voxel_side(old_voxel, change.owner.faceIndex)
    if pos is None:
        return
        
    # create new object
    voxel = Mesh(
        geometry = cube_geo,
        material = cube_mat,
        position = pos
    )
    
    # add new object to scene and object list
    pickable_objects.add(voxel)
    objects[tuple(map(int, voxel.position))] = voxel

In [ ]:
coord_html = HTML("Coords: ()")
def on_surf_mousemove(change):
    """
    Show rollOver helper on mousemove
    """
    rollOver_point.visible = True
    
    # convert to discrete coordinates
    pos = map_to_grid(change.owner.point)
    
    # update rollOver helper position
    rollOver_point.position = pos
    
    # write coordinates to html container
    coord_html.value = "Coords: (%d, %d, %d)" % tuple(pos)

def on_voxel_mousemove(change):
    """
    Show rollOver helper on mousemove on voxel
    """
    # convert to discrete coordinates
    voxel = change.owner.object
    pos = map_to_voxel_side(voxel, change.owner.faceIndex)
    if pos is None:
        rollOver_point.visible = False
        return
    rollOver_point.visible = True
    
    # update rollOver helper position
    rollOver_point.position = pos


In [ ]:
def on_click(change):
    if change.owner.object is surface:
        on_surf_click(change)
    else:
        on_voxel_click(change)
        
def on_mousemove(change):
    if change.owner.object is None:
        # Hide rollOver if outside
        rollOver_point.visible = False
    elif change.owner.object is surface:
        on_surf_mousemove(change)
    else:
        on_voxel_mousemove(change)

In [ ]:
click_picker.observe(on_click, names=['point'])
mousemove_picker.observe(on_mousemove, names=['faceIndex'])

### Camera and scene

In [ ]:
width = 800
height = 600

In [ ]:
camera = PerspectiveCamera(
    position = [500, 800, 1300],
    aspect = width/height,
    fov = 35, 
    near = 1,
    far = 10000
)
camera.lookAt([0,0,0])

In [ ]:
scene = Scene(
    children = [
        pickable_objects,
        surf_grid,
        rollOver_point,
        AmbientLight(
            color = '#606060'
        ),
        DirectionalLight(
            color = '#ffffff',
            position = normalize([1, 0.75, 0.5]),
            intensity = 0.5
        )
    ]
)

### Render the scene

In [ ]:
renderer = Renderer(
    camera = camera,
    scene = scene,
    controls = [
        OrbitControls(
            controlling = camera
        ),
        click_picker,
        mousemove_picker,
    ],
    background = '#f0f0f0',
    antialias = True,
    width=width,
    height=height,
)

In [ ]:
display(coord_html, renderer)